Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down to the end of the code to find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def matrix_widget(xMax_init = 10, yMax_init = 10, mode_init = 'A', A_base = [[1,0],[0,1]],
                  B_base = [[1,0],[0,1]], D_base = [[3,4,5],[-1,2,-1]], fixScale_init = False):
    
    # Declare widgets for interactive input
    boxLayout = {'width': '175px'}
    
    style = {'description_width': 'initial'}
    xMax_slider = widgets.IntSlider(min=0.1,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $x$:',
                                 value = xMax_init,
                                 continuous_update =True)
    yMax_slider = widgets.IntSlider(min=0.1,
                                 max=1000,
                                 step=1,
                                 description=r'Maximum $y$:',
                                 value = yMax_init,
                                 continuous_update =True)
    mode_list = widgets.Dropdown(options=['A','B', 'AB', 'BA'],
                                value = mode_init,
                                description=r'Transformation$:',
                                disabled=False)
    
    a1_box = widgets.FloatText(description= r'$a_{1,1}$:', value = A_base[0][0], 
                               layout=boxLayout)
    a2_box = widgets.FloatText(description= r'$a_{1,2}$:', value = A_base[0][1], 
                               layout=boxLayout)
    a3_box = widgets.FloatText(description= r'$a_{2,1}$:', value = A_base[1][0], 
                               layout=boxLayout)
    a4_box = widgets.FloatText(description= r'$a_{2,2}$:', value = A_base[1][1], 
                               layout=boxLayout)

    b1_box = widgets.FloatText(description= r'$b_{1,1}$:', value = B_base[0][0], 
                               layout=boxLayout)
    b2_box = widgets.FloatText(description= r'$b_{1,2}$:', value = B_base[0][1], 
                               layout=boxLayout)
    b3_box = widgets.FloatText(description= r'$b_{2,1}$:', value = B_base[1][0], 
                               layout=boxLayout)
    b4_box = widgets.FloatText(description= r'$b_{2,2}$:', value = B_base[1][1], 
                               layout=boxLayout)
    
    d1_box = widgets.FloatText(description= r'$d_{1,1}$:', value = D_base[0][0], 
                               layout=boxLayout)
    d2_box = widgets.FloatText(description= r'$d_{1,2}$:', value = D_base[0][1], 
                               layout=boxLayout)
    d3_box = widgets.FloatText(description= r'$d_{1,3}$:', value = D_base[0][2], 
                               layout=boxLayout)
    d4_box = widgets.FloatText(description= r'$d_{2,1}$:', value = D_base[1][0], 
                               layout=boxLayout)
    d5_box = widgets.FloatText(description= r'$d_{2,2}$:', value = D_base[1][1], 
                               layout=boxLayout)
    d6_box = widgets.FloatText(description= r'$d_{2,3}$:', value = D_base[1][2], 
                               layout=boxLayout)
    
    fixScale_check = widgets.Checkbox(value = fixScale_init,
                                   description='Fix 2nd axis',
                                   disabled=False,
                                   indent=True)
    
    # Link widgets as required
    widgets.jslink((xMax_slider,'value'),(yMax_slider,'value'))

    def matrix_plot(xMax, yMax, mode, a1, a2 ,a3 ,a4 , b1, b2, b3, b4, 
                    d1, d2, d3, d4, d5, d6, fixScale):
        
        # Package inputs into lists
        A_base = [[a1,a2],
                  [a3,a4]]
        B_base = [[b1,b2],
                  [b3,b4]]
        D_base = [[d1,d2,d3],
                  [d4,d5,d6]]

        # Convert lists to Numpy matrices
        A = np.asmatrix(A_base)
        B = np.asmatrix(B_base)
        D = np.asmatrix(D_base)

        # Setup tranformation matrix T, get determinant of T
        if mode == 'A':
            T = A
        elif mode == 'B':
            T = B
        elif mode == 'AB':
            T = A*B
        elif mode == 'BA':    
            T= B*A
        Det = np.linalg.det(T)
        
        # Apply matrix to get transformed data
        R = T*D
        
        # Convert back to array for plotting
        Dplot = np.asarray(D_base)
        Rplot = np.asarray(R)

        # Find scale of x/y axes for both plots. Fix if required
        if fixScale is True or Det == 0:
            xMaxTrans = xMax
            yMaxTrans = yMax
            xGridMax = xMax
            yGridMax = yMax
        else:
            xMaxTrans = (abs(Det)**0.5)*xMax
            yMaxTrans = (abs(Det)**0.5)*xMax
            xGridMax = max(xMax,xMaxTrans)
            yGridMax = max(yMax,yMaxTrans)

        # Generate grids 
#         gridStep = np.floor(max(xGridMax,yGridMax)/10)
        gridStep = max(xGridMax,yGridMax)/10
        NxGrids = int(xGridMax/gridStep)
        xGrid = []
        xGridTransformed = []
        currX = gridStep
        for i in range(NxGrids):
            for sig in [-1,1]:
                line = [[sig*currX,sig*currX],
                        [-yGridMax-gridStep/2,yGridMax+gridStep/2]]
                lineTrans = T*np.asmatrix(line)

                xGrid.append(line)
                xGridTransformed.append(np.asarray(lineTrans))

            currX += gridStep

        NyGrids = int(yGridMax/gridStep)
        yGrid = []
        yGridTransformed = []
        currY = gridStep
        for i in range(NyGrids):
            for sig in [-1,1]:
                line = [[-xGridMax-gridStep/2,xGridMax+gridStep/2],
                        [sig*currY,sig*currY]]
                lineTrans = T*np.asmatrix(line)

                yGrid.append(line)
                yGridTransformed.append(np.asarray(lineTrans))

            currY += gridStep

        # Transform axis for plot
        axisTransformed = []
        axisStyle = ['g--','b--']
        axisLabel = ['Original $x$ axis', 'Original $y$ axis']
        xAxis = [[-xGridMax-gridStep/2,xGridMax+gridStep/2],[0,0]]
        yAxis = [[0,0], [-yGridMax-gridStep/2,yGridMax+gridStep/2]]
        xAxisTrans = T*np.asmatrix(xAxis)
        yAxisTrans = T*np.asmatrix(yAxis)
        axisTransformed.append(np.asarray(xAxisTrans))
        axisTransformed.append(np.asarray(yAxisTrans))

        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(20,10))

        # plot gridlines
        for line in xGrid:
            ax[0].plot(line[0],line[1],'k',alpha = 0.2)

        for line in yGrid:
            ax[0].plot(line[0],line[1],'k',alpha = 0.2)

        # plot data
        ax[0].scatter(Dplot[0,:],Dplot[1,:], s=mrkrSize, c='r',alpha = 0.6,
          label = r'Initial data $D$' )
        ax[0].plot(Dplot[0,:],Dplot[1,:],'r',alpha = 0.6)
        ax[0].fill(Dplot[0,:],Dplot[1,:],'r',alpha = 0.2)

        # Add legend and format axes to look nice
        ax[0].axis('equal')
        ax[0].legend(loc='upper left', frameon=False,prop={'size':20})
        ax[0].autoscale(enable=True, axis='both', tight=True)
        ax[0].set_ylim(top = yMax, bottom = -yMax)
        ax[0].set_xlim(right = xMax, left = -xMax)
        ax[0].spines['bottom'].set_position('zero')
        ax[0].spines['top'].set_visible(False)
        ax[0].spines['left'].set_position('zero')
        ax[0].spines['right'].set_visible(False)
        ax[0].set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[0].set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[0].plot(1, 0, ">k", transform=ax[0].get_yaxis_transform(), clip_on=False)
        ax[0].plot(0, 1, "^k", transform=ax[0].get_xaxis_transform(), clip_on=False)
        ax[0].tick_params(labelsize=20)

        # plot transformed gridlines
        for line in xGridTransformed:
            ax[1].plot(line[0,:],line[1,:],'k',alpha = 0.2)

        for line in yGridTransformed:
            ax[1].plot(line[0,:],line[1,:],'k',alpha = 0.2)

        for line in axisTransformed:
            ax[1].plot(line[0,:],line[1,:],axisStyle.pop(0),alpha = 0.6, 
                       linewidth=2, label = axisLabel.pop(0))

        # Plot transformed data
        ax[1].scatter(Rplot[0,:],Rplot[1,:], s=mrkrSize, c='r',alpha = 0.6,
          label = r'Transformed data $'+mode+'D$' )
        ax[1].plot(Rplot[0,:],Rplot[1,:],'r',alpha = 0.6)
        ax[1].fill(Rplot[0,:],Rplot[1,:],'r',alpha = 0.2)

        ax[1].annotate(r'$|'+mode + '| = {:.4f}$'.format(Det),[0.75,0], 
                       xytext = [0.55,0.05], xycoords ='axes fraction', 
                       fontsize = 25, clip_on = True)

        # Add legend and format axes to look nice
        ax[1].axis('equal')
        ax[1].legend(loc='upper left', frameon=False,prop={'size':20})
        ax[1].autoscale(enable=True, axis='both', tight=True)
        ax[1].set_ylim(top = yMaxTrans, bottom = -yMaxTrans)
        ax[1].set_xlim(right = xMaxTrans, left = -xMaxTrans)
        ax[1].spines['bottom'].set_position('zero')
        ax[1].spines['top'].set_visible(False)
        ax[1].spines['left'].set_position('zero')
        ax[1].spines['right'].set_visible(False)
        ax[1].set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax[1].set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax[1].plot(1, 0, ">k", transform=ax[1].get_yaxis_transform(), clip_on=False)
        ax[1].plot(0, 1, "^k", transform=ax[1].get_xaxis_transform(), clip_on=False)
        ax[1].tick_params(labelsize=20)

        fig.subplots_adjust(hspace=0.4, wspace=0.6)
        plt.tight_layout()
    
    out = widgets.interactive_output(matrix_plot, {'xMax': xMax_slider,
                                                   'yMax': yMax_slider,
                                                   'mode': mode_list, 
                                                   'a1': a1_box,
                                                   'a2': a2_box,
                                                   'a3': a3_box,
                                                   'a4': a4_box,
                                                   'b1': b1_box,
                                                   'b2': b2_box,
                                                   'b3': b3_box,
                                                   'b4': b4_box,
                                                   'd1': d1_box,
                                                   'd2': d2_box,
                                                   'd3': d3_box,
                                                   'd4': d4_box,
                                                   'd5': d5_box,
                                                   'd6': d6_box,                                                       
                                                   'fixScale': fixScale_check})
    
    output = widgets.VBox([out,
                  widgets.HBox([xMax_slider,
                                yMax_slider,
                                fixScale_check]),
                  widgets.HBox([
                        widgets.HBox([widgets.VBox([a1_box,a3_box]),
                                      widgets.VBox([a2_box,a4_box])]),
                        widgets.HBox([widgets.VBox([b1_box,b3_box]),
                                      widgets.VBox([b2_box,b4_box])])
                                ]),
                        widgets.HBox([widgets.VBox([d1_box,d4_box]),
                                      widgets.VBox([d2_box,d5_box]),
                                      widgets.VBox([d3_box,d6_box]),
                                      mode_list])
                          ])
    
    display(output)


In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Deep(er) Dive: The beautiful geometry of matrices

As this is the first 'deeper dive' notebook, let me begin by explaining them a bit for you. When you teach (or learn) maths, it often is the case that you don't cover all the angles of a given concept, either because it's not on the curriculum, or because it is a bit too advanced. Unfortunately, it is also often the case that these extra concepts are *really* useful for truly understanding a topic, because they answer the 'why' questions I refer to in the videos. So I've decided not to do the same as everyone else and and show you these deeper concepts, precisely because they give you insights into what is truly happening behind the scenes. Matrices are a perfect place to start. Don't worry, **none of the deeper dives are in the assessment**, and we will not expect you to master this stuff in any way. What we would like you to do is give it a try, so that you can take a peek behind the curtain and maybe, if I get my material right, marvel at the beauty of what is behind.

#### Matrices can transform space!!

Like everything we have seen until now, matrices have a geometric interpretation. This should not be very surprising: we have seen that matrices are related to systems of equations, and we have also seen that systems of equations have a geometric counterpart in Cartesian space.

So matrix algebra (the rather dry stuff we saw in the lecture videos) also has a geometric side. As I said, no real surprise. What is more surprising, as you hopefully will see, is that the geometric interpretation of matrix algebra is very different from what we have seen until now. Matrices open the door to something very deep and rather beautiful: matrices don't just encode locations in $(x,y)$ space (the way a linear function does), they can also encode *transformations of space itself*. This notebook will illustrate this step by step, allowing to visualise this deep property of matrices and also understand the 'dry' algebra a bit better.

Let's start with a simple example using the stuff we covered in the videos. Suppose you have a matrix $A$ as defined below and a point with coordinates $x=3$, $y=-1$. These coordinates are gathered into a column vector ${\rm x}$.

$$A=\left( {\begin{array}{cc}
   \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \\
   -\frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \\
 \end{array} } \right) \qquad {\rm x} =\left( {\begin{array}{c}
   3 \\
   -1 \\
 \end{array} } \right)$$

If you think that I've picked the entries of $A$ in a very specific way, you'd be correct, but more on that in a second. Using matrix algebra, we know we can multiply $A$ and ${\rm x}$. Because the dimensions of $A$ are $2\times 2$ and ${\rm x}$ is  $2\times 1$ the result of the multiplication is also a $2\times 1$ vector.

$$d = A {\rm x}=\left( {\begin{array}{cc}
   \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \\
   -\frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \\
 \end{array} } \right) \left( {\begin{array}{c}
   3 \\
   -1 \\
 \end{array} } \right) = \left( {\begin{array}{c}
     3\frac{\sqrt{2}}{2} - \frac{\sqrt{2}}{2}\\
    -3\frac{\sqrt{2}}{2} - \frac{\sqrt{2}}{2}\\
 \end{array} } \right) = \left( {\begin{array}{c}
      \sqrt{2}\\
    -2\sqrt{2}\\
 \end{array} } \right)$$

In the slides, we saw that this matrix equation, $A{\rm x}=d$, can encode a system of equations. The ${\rm x}$ vector contains our two unknowns $x$ and $y$, the $d$ vector is the right hand side of the system. Suppose for a moment we didn't know that $x=3$ and $y=-1$, we could write:

$$\left\{\begin{aligned}
\frac{\sqrt{2}}{2}x - \frac{\sqrt{2}}{2}y & = \sqrt{2}\\
-\frac{\sqrt{2}}{2}x + \frac{\sqrt{2}}{2}y & = -2\sqrt{2}\\
\end{aligned}\right.$$

In this case, remember that $d$ is the information we have and ${\rm x}$ is the information we want.

So far, you might say that we've not really done any geometry. What we have done is picked a point in $(x,y)$ space, defined by a column vector ${\rm x}$ and used the matrix $A$ to calculate the coordinates of a new point defined by another column vector $d=A{\rm x}$. In our examples we picked a specific set of numbers, but hopefully you can see that we could have picked ***any*** number for $x$ or $y$

In fact, if we had plenty of time, we could perform this calculation for a set of $N$ points and get a new set of $N$ points. The diagram below shows you what this would look like for 3 points, gathered together into a $2\times 3$ matrix $D$. 

$$D=\left( {\begin{array}{ccc}
   3 & 4 & 5\\
   -1 & 2 & -1 \\
 \end{array} } \right)$$

Note that the first column vector in this matrix corresponds to the coordinates for the point we chose above. The second and third column vector contain the coordinates of two more point. The diagram on the left shows you the original coordinates of the 3 points, with the resulting triangle 'colored in' and the diagram on the left shows you the coordinates of the points you get when you multiply by the matrix $A$.

- **Note**: in reality, thanks to the dark art of computer programming, the diagram below shows you *more* than 3 points: it calculates the result of the matrix multiplication for all the intersections of the grid lines. It's actually the gridlines that are really important in allowing to 'see' the effect of the matrix.


In [3]:
matrix_widget(A_base = [[0.7071,0.7071],[-0.7071,0.7071]])

It should be pretty obvious that the diagram on the right is simply the diagram on the left rotated clockwise by 45°. If you have the courage and you want to check this, feel free to calculate the coordinates of the last two points with matrix multiplication (we already did the first point, use it as an example!) and plot them on a piece of paper. Please also feel free to change the locations of the 3 points in the $D$ matrix: whatever points you try, they will just be rotated clockwise by 45°.

Let's stop for a second, step back, and let the implication sink in. The geometric interpretation of the matrix $D$ should be straightforward: it just encodes the coordinate of points in $(x,y)$ space. The geometric interpretation of $A$, however is much deeper: it encodes a ***transformation of space itself***, as is visible from the fact all the original gridlines are affected. 

As you may have guessed, the numbers I picked for this example were specifically chosen to ensure that $A$ would be a **rotation** matrix. In general, if you want to rotate the space clockwise by an angle $\theta$, the entries in your matrix are given by the sine and cosine of that angle (don't worry about this is your trigonometry is rusty):

$$A=\left( {\begin{array}{cc}
   \cos \theta & \sin \theta \\
   -\sin \theta & \cos \theta \\
 \end{array} } \right)$$

As you may be starting to suspect other transformations of space can be carried out by picking the right matrix. A few examples of simple transformations are provided below. Feel free to try these in the diagram above by replacing the relevant entries of $A$. 
- A **scaling** matrix increases or reduces the size of a unit of space. You can see below that it is just the identity matrix multiplied by a number $a$. If $a>1$, the plot scales up, if $a<1$ it shrinks (use the 'fix 2nd axis' check box to see this).

$$A=\left( {\begin{array}{cc}
   a & 0 \\
   0 & a \\
 \end{array} } \right)$$

- A **symmetry** matrix flips the space around a line. In principle you can design one for any line in the $(x,y)$ space, but to keep things simple we'll look at symmetry around the axes. $A$ generates a symmetry around the $y$ axis, and $A'$ around the $x$ axis.

$$A=\left( {\begin{array}{cc}
   -1 & 0 \\
   0 & 1 \\
 \end{array} } \right) \qquad A'=\left( {\begin{array}{cc}
   1 & 0 \\
   0 & -1 \\
 \end{array} } \right)$$

- **Squeezing/stretching** matrices are a bit like scaling matrices, but the two axes are scaled by different amounts. If either $a=1$ or $b=1$, then you have a **stretch** matrix, if $b=1/a$ you have a squeeze matrix.

$$A=\left( {\begin{array}{cc}
   a & 0 \\
   0 & b \\
 \end{array} } \right)$$

- A **Shear** matrix will slant the the diagram to one side by an amount determined by $a$ (the larger $a$ the larger the effect). $A$ will slant the diagram parallel to the $x$ axis, and $A'$ parallel to the $y$ axis.

$$A=\left( {\begin{array}{cc}
   1 & a \\
   0 & 1 \\
 \end{array} } \right) \qquad A'=\left( {\begin{array}{cc}
   1 & 0 \\
   a & 1 \\
 \end{array} } \right)$$

An important aspect of all these examples is that while the coordinates of the red points are moved around in different ways by the different matrices, intuitively, you should see that using the 'old' grid and old axes you can always recover the original coordinates. This is the idea behind matrix inversion: when you get given a system of equations $A{\rm x}=d$, $d$ corresponds to a point in the right hand side diagram (the data you *have*), and $x$ is a point in the left hand side diagram (the data you *want*). The inversion of matrix $A$, essentially tells you where the original axes are in the transform ed diagram, allowing you to find the original, pre-transformation coordinates of the point.

#### Combining transformations

A really powerful aspect of this geometric interpretation of matrices is that its helps you visualise matrix multiplication as the action of combining transformations. This is illustrated in the diagram below, where $A$ is the original 45° rotation and $b$ is a shear matrix parallel to the $x$. The transformation that results from the product the two matrices, $AB$, is simply the rotation followed by the shear. Feel free to combine the simple transformations listed above!

This example should help you to visualise two important concepts:

1. First, you should be able to appreciate that pretty much ***any*** transformation could be obtained by combining lots of individually simple transformation matrices, so you could generate lots of complex transformations this way.

2. Second, this should help you understand why matrix multiplication does not *commute*, in other words why $AB \ne BA$ (use the drop-down list to try the two different orders). When you have a combination of transformations (say $AB$) applied to some data $D$, i.e you calculate $ABD$, the order in which the transformations are applied is right-to-left. You take $D$, then you apply the transformation $B$ then finally the transformation $A$. The order in which you carry out the transformations matters and in general applying $B$ first then $A$ will give a different result to $A$ first then $B$.


In [4]:
matrix_widget(A_base = [[0.7071,0.7071],[-0.7071,0.7071]], B_base = [[1,1.25],[0,1]],mode_init = 'AB')

#### Visualising the determinant.

Up until now, most of the the basic examples provided for you to try have displayed $|A|=1$ or $|B|=1$ in the diagrams, so I haven't really had to comment on the role of the determinant of matrices $A$ and $B$. One notable example is the case of the **scaling** matrix, which we will focus on in the diagram below. Just as a quick reminder, this is given by:

$$A=\left( {\begin{array}{cc}
   a & 0 \\
   0 & a \\
 \end{array} } \right)$$

The determinant of $A$ is easy to calculate. Using the formula from the lecture material, you can see that $|A|=a^2$. In the case of the diagram below, with $a=2$, the determinant is 4. It should also be clear that this scaling matrix increases the size of the red area. But by how much? Well, by using $a=2$ and scaling the space up, we are doubling every length in the diagram (such as the base and height of the red triangle). Doubling the lengths, however, means quadrupling surfaces: The surface area of the red  triangle in the diagram on the right is four times that of the one on the left. 

In [5]:
matrix_widget(A_base = [[2,0],[0,2]], mode_init = 'A', fixScale_init = True)

It is no coincidence that the value of the determinant $|A|=4$ corresponds to how much areas are increased by in the right-hand side diagram. In fact, this is precisely what the determinant measures. It is also why we use the vertical bar notation $|A|$ for the determinant, which some of you might remember means **absolute value** for scalars (good old regular numbers). This is because the determinant is a *generalisation* of the concept of absolute value. A scalar $a$ can be thought of as a $1\times 1$ matrix, and if you multiply an unknown number $x$ by $a$, the absolute value $|a|$ tells you how much this one dimensional number (a length) has changed by. For a $2\times 2$ matrix $A$, $|A|$ tells us how the 2D area is scaled. For a $3\times 3$ matrix, $|A|$ tells us how the 3D volume is scaled. The logic continues beyond 3, but at that  point we refer to 'hypervolumes' because we can't visualise the geometry. No need to worry about this, my point was just to show the idea...

Try inputting $a=0.5$ to see the opposite effect: the determinant now equal to 0.25 and the red area has shrunk to one quarter of its original size.

#### Why you can't invert a matrix if its determinant is equal to 0

So the determinant of a transformation tells us how much areas (or volumes if you have more dimensions) are scaled by when you apply the transformation. So what would a 0 determinant mean, from a geometrical point of view? Well, clearly, if we use the logic I talked about, wouldn't this mean that in our $2\times 2$ example all areas would be shrunk to zero? In fact this is exactly what happens! See the example below: I have picked a matrix $A$ with $|A|=0$ and you can see that the original two-dimensional $(x,y)$ Cartesian space has collapsed to a one-dimensional space: a line. Any point you could have picked anywhere in the original space will get squeezed onto that line. The concept of an *area* or a *surface* does not exist for line, therefore the transformation generated by matrix $A$ squeezes any pre-existing area to zero.


In [6]:
matrix_widget(A_base = [[0.2,0.3],[0.4,0.6]], mode_init = 'A', fixScale_init = True)

If you remember the discussion above, we saw that the geometrical reason why you can 'undo' the multiplication by a matrix $A$ when you do matrix inversion is that you can always use the location of the 'old' axes and grid lines to find the coordinates of the points before the space was transformed. In this case, our 2-dimensional space has collapsed into a 1-dimensional space, and you can no longer do this. In the transformed picture, both the original $x$ and $y$ as well as all the old gridlines get squeezed on top of each other onto the single 1-dimensional line that is our new space. Given that all these lines are now on top of each other, we can't count the grildlines parallel to the old $x$ or $y$ axis to get the coordinates: they don't exist anymore!

#### Wrapping up

For those of you that have made it this far, I hope the payoff is worth the effort: You should now have a much better understanding of what matrices actually do, and why some of their funny properties (like $AB \ne BA$) are what they are. As I keep explaining, algebra is useful when you need to calculate things, but often, it is the geometric connection that helps you truly visualise and understand what is really going on.
